In [14]:
import random

from IPython.display import IFrame
IFrame("Blatt04.pdf", width=1000, height=500)

In [15]:
def discrete_log(base, modulo):
    return [(x, (base ** x) % modulo) for x in range (modulo)]
    

print(discrete_log(2, 13))

[(0, 1), (1, 2), (2, 4), (3, 8), (4, 3), (5, 6), (6, 12), (7, 11), (8, 9), (9, 5), (10, 10), (11, 7), (12, 1)]


# 4.2

m' with Alice key
m'' with Alice and Bob keys
m''' with Bob key

m' XOR m'' XOR m''' = m

In [16]:
def shamir_xor(m=10):
    alice_key = random.randint(1,10)
    bob_key = random.randint(1,10)
    m1 = m ^ alice_key
    m2 = m1 ^ bob_key
    m3 = m2 ^ alice_key
    m4 = m3 ^ bob_key
    m5 = m1 ^ m2 ^ m3
    return m, alice_key, bob_key, m1, m2, m3, m4, m5


print(shamir_xor())

(10, 1, 4, 11, 15, 14, 10, 10)


In [17]:
def mod_exp(a, x, n):
    d = 1
    for i in bin(x)[2:]:
        d = (d * d) % n
        if int(i) == 1: d = (d * a) % n
    return d


def diffie_hellman(n, a, x, y):
    x1 = mod_exp(a, x, n)
    print(f"Alice calculates x1 = ({a}^{x} mod {n}) = {x1} and sends it to Bob.")
    y1 = mod_exp(a, y, n)
    print(f"Bob calculates y1 = ({a}^{y} mod {n}) = {y1} and sends it to Alice.")
    za = mod_exp(y1, x, n)
    print(f"Alice calculates the key z = ({y1}^{x} mod {n}) = {za}")
    zb = mod_exp(x1, y, n)
    print(f"Bob calculates the key z = ({x1}^{y} mod {n}) = {zb}")
    
    return za, zb


diffie_hellman(17, 4, 3, 6)

Alice calculates x1 = (4^3 mod 17) = 13 and sends it to Bob.
Bob calculates y1 = (4^6 mod 17) = 16 and sends it to Alice.
Alice calculates the key z = (16^3 mod 17) = 16
Bob calculates the key z = (13^6 mod 17) = 16


(16, 16)

In [30]:
def extggT(a,b):
    if b == 0: return a,1,0
    d,x,y = extggT(b, a % b)
    return d,y,x-(a//b)*y


def el_gamal(n, a, y, m):
    y1 = mod_exp(a, y, n)
    correct = []
    for i in range(n):
        x = mod_exp(a, i, n)
        za = mod_exp(y1, i, n)
        c = (m * za) % n
        zb = mod_exp(x, y, n)
        zb_inv = extggT(zb, n)[1] % n
        mb = (c * zb_inv) % n
        correct.append((x, c))
    return correct
        

el_gamal(11, 6, 2, 7)

[(1, 7),
 (6, 10),
 (3, 8),
 (7, 2),
 (9, 6),
 (10, 7),
 (5, 10),
 (8, 8),
 (4, 2),
 (2, 6),
 (1, 7)]

In [31]:
def is_square_rest(a, n):
    if mod_exp(a, (n - 1) // 2, n) == 1: return True
    return False


def mod_square_root(a, n):
    if n % 4 != 3:
        raise Exception(f"{n} is not a blum-prime")
    b = mod_exp(a, (n + 1) // 4, n)
    return b, n-b


print(is_square_rest(8, 103))
print(mod_square_root(8, 103))

True
(76, 27)


In [39]:
import math


def chinese_remainder(a, n):
    length = len(n)
    n_prod = math.prod(n)
    m = [n_prod // i for i in n]
    y = [extggT(m[i], n[i])[1] % n[i] for i in range(length)]
    x = sum([(a[i] * y[i] * m[i]) % n_prod for i in range(length)])
    print(m, y)
    return x


def rabin(p, q, m):
    n = p * q
    c = (m ** 2) % n
    mp1, mp2 = mod_square_root(c, p)
    mq1, mq2 = mod_square_root(c, q)
    print(mp1, mp2, mq1, mq2)
    x1 = chinese_remainder([mp1, mq1], [p, q]) % n
    x2 = chinese_remainder([mp1, mq2], [p, q]) % n
    x3 = chinese_remainder([mp2, mq1], [p, q]) % n
    x4 = chinese_remainder([mp2, mq2], [p, q]) % n
    return x1, x2, x3, x4


print(rabin(31, 23, 111))

18 13 4 19
[23, 31] [27, 3]
[23, 31] [27, 3]
[23, 31] [27, 3]
[23, 31] [27, 3]
(142, 111, 602, 571)


In [21]:
n = 31*23
a = 200
for x in range(n):
    if (x**2) % n == a: print(x, n-x)

111 602
142 571
571 142
602 111
